In [1]:
import pandas as pd

In [27]:
df=pd.read_csv(r'E:\Data Analysis\GemstonePrice\notebooks\data\Diamond.csv')

In [29]:
df.duplicated().sum()

np.int64(0)

In [28]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,0.00,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df=df.drop(labels='id',axis=1)

In [5]:
df['z']=df['z'].replace(" ",'0')

In [6]:
df['z']=df['z'].astype(float)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,0.00,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
df.duplicated().sum()

np.int64(31)

In [8]:
## Independent and dependent featuers
X=df.drop(labels='price',axis=1)
y=df[['price']]


In [9]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [10]:
## segretting categorical and numerical columns
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns


In [11]:
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.preprocessing import OrdinalEncoder

## Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    
    ]
)

## Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
 
    ]
)

preprocessor=ColumnTransformer(
    [
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)

In [14]:
## Train test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)


In [15]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [16]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-0.005148,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.843933,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,1.168582,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-2.052931,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.878120,0.874076,2.144670,1.352731


In [17]:
# Model Training
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error



In [18]:
regression =LinearRegression()
regression.fit(X_train,y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [19]:
regression.coef_

array([[ 6.43370657e+03, -1.38682448e+02, -7.04422300e+01,
        -1.75698985e+03, -5.25743586e+02,  6.04164849e-01,
         7.24212076e+01, -4.60399656e+02,  6.50713706e+02]])

In [20]:
regression.intercept_

array([3970.76628955])

In [21]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [22]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [23]:
## train multiple models
models={
    'LinearRegression':LinearRegression(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'ElasticNet':ElasticNet()

}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    
    
    # training the model
    model.fit(X_train,y_train)
    
    # predicting the model
    y_pred=model.predict(X_test)
    mae, rmse ,r2_square=evaluate_model(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Performance on Test Set")
    print("RMSE:" ,rmse)
    print("MAE:" ,mae)
    print("R2_Score:" ,r2_square)

    r2_list.append(r2_square)

    print("="*35)
    print("\n")

LinearRegression
Model Performance on Test Set
RMSE: 1014.0061211200209
MAE: 674.1092686229048
R2_Score: 0.9368781997494069


Ridge
Model Performance on Test Set
RMSE: 1014.008174511281
MAE: 674.1402127903685
R2_Score: 0.9368779441022633


Lasso
Model Performance on Test Set
RMSE: 1013.9533816591793
MAE: 675.1832015499325
R2_Score: 0.9368847656331243


ElasticNet
Model Performance on Test Set
RMSE: 1558.9976530970523
MAE: 1055.5964993055686
R2_Score: 0.8507930179787538




In [24]:
from sklearn.model_selection import GridSearchCV
def evalute_metrics(true,predict):
    score=r2_score(true,predict)
    rmse=np.sqrt(mean_squared_error(true,predict))
    mae=mean_absolute_error(true,predict)

    return score,mae,rmse



def evalute_model(X_train,y_train,X_test,y_test,models,param):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        para=param[list(models.keys())[i]]

        gs=GridSearchCV(model,param_grid=para,cv=3,n_jobs=-1)
        gs.fit(X_train,y_train)

        model.set_params(**gs.best_params_)
        model.fit(X_train,y_train)

        #y_train_pred=model.predict(X_train)
        y_test_pred=model.predict(X_test)
        score,mae,rmse=evalute_metrics(y_test,y_test_pred)

        test_model_score=score
        report[list(models.keys())[i]]=test_model_score
        
    return report
   

In [ ]:
import warnings
warnings.filterwarnings('ignore')

models={
                "Linear Regression":LinearRegression(),
                "Lasso":Lasso(),
                "Ridge":Ridge(),
                "DecisionTree Regeressor":DecisionTreeRegressor(),
                "RandomForest Regressor":RandomForestRegressor(),
                "AdaBoost Regressor":AdaBoostRegressor(),
                "GradiantBoost Regressor":GradientBoostingRegressor(),
                "XGB Regressor":XGBRegressor(),
                "CatBoost Regressor":CatBoostRegressor()
            }

param = {
           "Linear Regression": {
            #"fit_intercept": [True, False]
            },

            "Lasso": {
                #"alpha": [0.001, 0.01, 0.1, 1, 10],
                #"max_iter": [1000, 5000]
            },

            "Ridge": {
                #"alpha": [0.1, 1, 10, 50],
                #"solver": ["auto", "svd", "cholesky"]
            },
            "DecisionTree Regeressor": {
                #"criterion": ["squared_error", "friedman_mse"],
                #"max_depth": [None, 5,3, 10],
                # "min_samples_split": [2, 5, 10],
                #"min_samples_leaf": [1, 2, 4]
            },

            "RandomForest Regressor": {
                #'n_estimators': [100, 200, 300],
                #'max_depth': [None, 10, 20, 30],
                #'max_features': ['sqrt', 'log2'],
                #'bootstrap': [True, False]
            },

            "AdaBoost Regressor": {
                #"n_estimators": [50, 100, 200],
                #"learning_rate": [0.01, 0.1, 1]
            },

            "GradiantBoost Regressor": {
                #"n_estimators": [100, 200],
                #"learning_rate": [0.01, 0.1],
                #"max_depth": [3, 5],
                #"subsample": [0.8, 1.0]
            },

            'XGB Regressor' : {
                #"n_estimators": [200, 400],
                #"learning_rate": [0.03, 0.05, 0.1],
                #"max_depth": [4, 6, 8],
                #"subsample": [0.7, 0.8, 1.0],
                #"colsample_bytree": [0.7, 0.8, 1.0],
                #"reg_lambda": [1, 5, 10]    
            },
            'CatBoost Regressor':{
                #"iterations": [300, 500],
                #"learning_rate": [0.03, 0.05, 0.1],
                #"depth": [4, 6, 8],
                #"l2_leaf_reg": [1, 3, 5, 7]
            }    
        }


model_report:dict=evalute_model(X_train,y_train,X_test,y_test,models,param)
best_model_score=max(sorted(model_report.values()))

best_model_name=list(model_report.keys())[list(model_report.values()).index(best_model_score)]

best_model=models[best_model_name]
print(f"Best Model Found , Model Name : {best_model_name} , R2 Score : {best_model_score}")
print("\n====================================================================================\n")


Learning rate set to 0.088926
0:	learn: 3706.4982523	total: 152ms	remaining: 2m 31s
1:	learn: 3413.8529656	total: 162ms	remaining: 1m 20s
2:	learn: 3141.6797119	total: 174ms	remaining: 57.9s
3:	learn: 2894.0219261	total: 184ms	remaining: 45.9s
4:	learn: 2664.6723410	total: 197ms	remaining: 39.1s
5:	learn: 2464.5446919	total: 209ms	remaining: 34.7s
6:	learn: 2280.8533541	total: 226ms	remaining: 32.1s
7:	learn: 2112.4025773	total: 244ms	remaining: 30.3s
8:	learn: 1959.0003247	total: 261ms	remaining: 28.8s
9:	learn: 1820.1531247	total: 274ms	remaining: 27.1s
10:	learn: 1698.5354646	total: 285ms	remaining: 25.6s
11:	learn: 1589.2092515	total: 296ms	remaining: 24.4s
12:	learn: 1487.2952179	total: 307ms	remaining: 23.3s
13:	learn: 1395.8687046	total: 319ms	remaining: 22.5s
14:	learn: 1317.7875400	total: 331ms	remaining: 21.7s
15:	learn: 1242.9439896	total: 343ms	remaining: 21.1s
16:	learn: 1175.5706869	total: 355ms	remaining: 20.5s
17:	learn: 1114.1824797	total: 367ms	remaining: 20s
18:	lear